# Проект Дашборд конверсий

### Подготовим данные к работе. Импортируем библиотеку Pandas для загрузки и выгрузки данных в формате csv и библиотеку Requests для запроса данных по API

In [55]:
import pandas as pd
import requests

### Загрузим визиты и регистрации из csv файлов

In [56]:
df_visit = pd.read_csv("d:/2023/Аналитик данных/Модуль 3/Дашборд конверсий/visits_1k.csv")
df_visit.head(1000)

,uuid,platform,user_agent,date
0,1de9ea66-70d3-4a1f-8735-df5ef7697fb9,web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,2023-03-01T13:29:22
1,f149f542-e935-4870-9734-6b4501eaf614,web,Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) Apple...,2023-03-01T16:44:28
2,f149f542-e935-4870-9734-6b4501eaf614,web,Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) Apple...,2023-03-06T06:12:36
3,08f0ebd4-950c-4dd9-8e97-b5bdf073eed1,web,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109...,2023-03-01T20:16:37
4,08f0ebd4-950c-4dd9-8e97-b5bdf073eed1,web,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109...,2023-03-05T17:42:47
...,...,...,...,...
995,8bc047ac-dc03-4a61-8037-7371f729fa34,web,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15....,2023-03-02T23:57:06
996,8bc047ac-dc03-4a61-8037-7371f729fa34,web,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15....,2023-03-07T23:05:08
997,3f78ac76-6f81-43ec-85e8-f3cf74fc8fdc,web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,2023-03-02T12:47:09
998,3f78ac76-6f81-43ec-85e8-f3cf74fc8fdc,web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,2023-03-07T00:07:34


In [57]:
df_regist = pd.read_csv("d:/2023/Аналитик данных/Модуль 3/Дашборд конверсий/regs_1k.csv")
df_regist.head(1000)

,date,user_id,email,platform,registration_type
0,2023-03-01T00:25:39,8838849,joseph95@example.org,web,google
1,2023-03-01T14:53:01,8741065,janetsuarez@example.net,web,yandex
2,2023-03-01T14:27:36,1866654,robert67@example.org,web,google
3,2023-03-01T02:42:34,1577584,elam@example.net,web,apple
4,2023-03-01T10:27:14,4765395,stephanie68@example.net,web,yandex
...,...,...,...,...,...
995,2023-03-05T03:19:40,6414793,jerryrivera@example.com,android,google
996,2023-03-05T15:03:48,53167,laura65@example.net,android,google
997,2023-03-05T09:51:08,4969623,jasmine42@example.net,android,email
998,2023-03-05T16:25:59,7636873,imclaughlin@example.net,android,google


### Импортируем библиотеку json для работы с данными в формате json

In [58]:
import json

In [67]:
#Отправляем запрос по API для получения визитов по URL
visit_json = requests.get("https://data-charts-api.hexlet.app/visits?begin=2023-03-01&end=2023-09-01") 
data_visit_api = visit_json.json() #Преобразуем данные в объект JSON
df_visit_api = pd.DataFrame(data_visit_api) #преобразуем JSON в DataFrame
df_visit_api.head()
#df_visit_api.info

,datetime,platform,user_agent,visit_id
0,"Wed, 01 Mar 2023 23:17:37 GMT",web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,1de9ea66-70d3-4a1f-8735-df5ef7697fb9
1,"Sat, 04 Mar 2023 03:23:34 GMT",web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,1de9ea66-70d3-4a1f-8735-df5ef7697fb9
2,"Wed, 01 Mar 2023 12:57:13 GMT",web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,f149f542-e935-4870-9734-6b4501eaf614
3,"Wed, 01 Mar 2023 13:07:19 GMT",web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,08f0ebd4-950c-4dd9-8e97-b5bdf073eed1
4,"Sun, 05 Mar 2023 22:02:39 GMT",web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,08f0ebd4-950c-4dd9-8e97-b5bdf073eed1


In [71]:
#Отправляем запрос по API для получения регистраций по URL
regist_json = requests.get("https://data-charts-api.hexlet.app/registrations?begin=2023-03-01&end=2023-09-01") 
data_regist_api = regist_json.json() #Преобразуем данные в объект JSON
df_regist_api = pd.DataFrame(data_regist_api) #преобразуем JSON в DataFrame
df_regist_api.head()
df_regist_api.info

<bound method DataFrame.info of                             datetime                         email platform  \
0      Wed, 01 Mar 2023 13:21:20 GMT             jlane@example.net      web   
1      Wed, 01 Mar 2023 01:14:06 GMT       dyerkenneth@example.net      web   
2      Wed, 01 Mar 2023 19:47:32 GMT             kwade@example.com      web   
3      Wed, 01 Mar 2023 19:19:26 GMT       fisherbrian@example.net      web   
4      Wed, 01 Mar 2023 02:30:54 GMT         johngomez@example.org      web   
...                              ...                           ...      ...   
28192  Thu, 31 Aug 2023 20:51:43 GMT       lunazachary@example.com      ios   
28193  Thu, 31 Aug 2023 23:07:32 GMT        amyramirez@example.org      ios   
28194  Thu, 31 Aug 2023 14:56:54 GMT         webermark@example.org      ios   
28195  Thu, 31 Aug 2023 22:52:45 GMT  alexandermichael@example.com      ios   
28196  Thu, 31 Aug 2023 20:54:05 GMT            jose04@example.net      ios   

      registration_